In [100]:
import pandas as pd
import numpy as np
import pprint as pp
from collections import OrderedDict
import requests
import json
import datetime
import spacy
nlp = spacy.load('en_core_web_lg')

# File to Load (Remember to Change These)
file_to_load = "db/schoolShootingData_withGeoCoordinates.csv"
schoolData = pd.read_csv(file_to_load, encoding="utf-8-sig")
schoolData.reset_index()
schoolDataForNlp = schoolData[['School','Category','Summary','Narrative (Detailed Summary/ Background)']]
#Define 

#sandyHookNarrative = nlp("156 shots were fired in 5 minutes - shooter had multiple weapons with him, 10 mm Glock was found jammed next to the body, after he used to kill himself, 9mm Sig Sauer was never fired. Shooter reportedly shouted "Look at me" and was swearing. Leaked police investigation noted shooter had created a 7x4 spreadsheet listing top 500 mass killings with weapons used and scoresheet; he had removed the hard drive from his computer and damaged it prior to the shooting Shooter reloaded every time he entered a new room even if the clip was not empty. Shooter's mother was making straw purchases of guns for him, encouraging his gun obsession. Shooter shot his mother before he went to the school and finally he shot himself. Shooter was carrying his brother's ID and police initially thought he was the shooter. Police found a safe with over 1400 rounds of ammo in his home, multiple weapons, NRA certificate, and a photograph holding a gun to his head. Shooter had cut off contact with both his father and brother in the two years before the shooting and at one point communicated with his mother, who lived in the same house, only by email. He had his windows covered with black garbage bags. He was mostly living in isolation and played computer games. He was home-schooled since 8th grade. He was examined by several psychiatrist, took medication for a short time but was not open to therapy. He started editing Wikipedia entries on various well-known mass murderers.")
#sandyHookSummary = nlp ("Shooter though society was broken and mass shooting helps save children from negative influence")

In [101]:
schoolDataForNlp.head()

,School,Category,Summary,Narrative (Detailed Summary/ Background)
0,Hine Junior High School,Accidental,"Didn't know how to operate pistol, cocked hamm...",Student showing off gun cocked hammer and coul...
1,Sousa Junior High,Accidental,Occurred during horseplay in the school,"14YOM student shot during ""horseplay"" in the s..."
2,Unnamed High School,Robbery,Student shot at twice during attempted robbery...,Group of 10 teens attempted to rob 16YOM (Jame...
3,John F. Kennedy High School,Escalation of Dispute,Argument in school hallway escalated into shoo...,Argument between shooter and victim escalated ...
4,David Starr Jordan High School,Unknown,Teens fired at security guard who returned fir...,Group of teens fired shots at the night time s...


In [79]:
schoolDataForNlp['nlp summary'] = (schoolDataForNlp['Summary']).map(lambda x: nlp(x))
schoolDataForNlp['nlp Narrative'] = (schoolDataForNlp['Narrative (Detailed Summary/ Background)']).map(lambda x: nlp(x))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [80]:
schoolDataForNlp.head()

,School,Category,Summary,Narrative (Detailed Summary/ Background),nlp summary,nlp Narrative
0,Hine Junior High School,Accidental,"Didn't know how to operate pistol, cocked hamm...",Student showing off gun cocked hammer and coul...,"(Did, n't, know, how, to, operate, pistol, ,, ...","(Student, showing, off, gun, cocked, hammer, a..."
1,Sousa Junior High,Accidental,Occurred during horseplay in the school,"14YOM student shot during ""horseplay"" in the s...","(Occurred, during, horseplay, in, the, school)","(14YOM, student, shot, during, "", horseplay, ""..."
2,Unnamed High School,Robbery,Student shot at twice during attempted robbery...,Group of 10 teens attempted to rob 16YOM (Jame...,"(Student, shot, at, twice, during, attempted, ...","(Group, of, 10, teens, attempted, to, rob, 16Y..."
3,John F. Kennedy High School,Escalation of Dispute,Argument in school hallway escalated into shoo...,Argument between shooter and victim escalated ...,"(Argument, in, school, hallway, escalated, int...","(Argument, between, shooter, and, victim, esca..."
4,David Starr Jordan High School,Unknown,Teens fired at security guard who returned fir...,Group of teens fired shots at the night time s...,"(Teens, fired, at, security, guard, who, retur...","(Group, of, teens, fired, shots, at, the, nigh..."


In [95]:
suicide = nlp("suicide")
revenge = nlp("revenge")
feud = nlp("feud")
accident = nlp("accident")
fight = nlp("fight")

classification = ["suicide","revenge","feud","accident","fight"]

def classificationOfType(nlpData):
    dataScores =  [ suicide.similarity(nlpData),revenge.similarity(nlpData),
                    feud.similarity(nlpData),
                        accident.similarity(nlpData),fight.similarity(nlpData)]
    return classification[dataScores.index(max(dataScores))]
  

schoolDataForNlp['nlp summary scores'] = (schoolDataForNlp['nlp Narrative']).map(lambda x: classificationOfType(x))



/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [93]:
schoolDataForNlp['nlp summary scores'][15]


'fight'

In [58]:
schoolDataForNlp['Narrative (Detailed Summary/ Background)'][10]

'Father (Cuban exile) killed inside school while picking up child by 27 YOM gunman. First day the school was integrated.'

In [59]:
schoolDataForNlp['Summary'][10]

'Father (Cuban exile) killed by gunman at school'

In [71]:
schoolDataForNlp.head(2)

,School,Category,Summary,Narrative (Detailed Summary/ Background)
0,Hine Junior High School,Accidental,"Didn't know how to operate pistol, cocked hamm...",Student showing off gun cocked hammer and coul...
1,Sousa Junior High,Accidental,Occurred during horseplay in the school,"14YOM student shot during ""horseplay"" in the s..."


In [96]:
schoolDataForNlp.Category.unique()

array(['Accidental', 'Robbery', 'Escalation of Dispute', 'Unknown',
       'Gang-related', 'Mental Health', 'Racial',
       'Domestic w/ Targeted Victim', 'Officer Involved Shooting',
       'Anger Over Grade/Suspension/Discipline', 'Bullying',
       'Indiscriminate Shooting', 'Murder', 'Illegal Drug Related',
       'Murder/Suicide', 'Suicide/Attempted',
       'Intentional Property Damage', 'Self Defense', 'Hostage Standoff',
       'Murder/Suicide '], dtype=object)

In [97]:
categoriesOf = ['Accidental', 'Robbery', 'Escalation of Dispute', 'Unknown',
  'Gang-related', 'Mental Health', 'Racial',
  'Domestic w/ Targeted Victim', 'Officer Involved Shooting',
  'Anger Over Grade/Suspension/Discipline', 'Bullying',
  'Indiscriminate Shooting', 'Murder', 'Illegal Drug Related',
  'Murder/Suicide', 'Suicide/Attempted',
  'Intentional Property Damage', 'Self Defense', 'Hostage Standoff',
  'Murder/Suicide'
] 
def classificationOfType(nlpData):
    dataScores = [nlp(i).similarity(nlpData) for i in categoriesOf]
    return  categoriesOf[dataScores.index(max(dataScores))]

schoolDataForNlp['nlp summary scores With New Classification'] = (schoolDataForNlp['nlp summary']).map(lambda x: classificationOfType(x))



/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [99]:
schoolDataForNlp.head(20)

,School,Category,Summary,Narrative (Detailed Summary/ Background),nlp summary,nlp Narrative,nlp summary scores,nlp summary scores With New Classification
0,Hine Junior High School,Accidental,"Didn't know how to operate pistol, cocked hamm...",Student showing off gun cocked hammer and coul...,"(Did, n't, know, how, to, operate, pistol, ,, ...","(Student, showing, off, gun, cocked, hammer, a...",fight,Anger Over Grade/Suspension/Discipline
1,Sousa Junior High,Accidental,Occurred during horseplay in the school,"14YOM student shot during ""horseplay"" in the s...","(Occurred, during, horseplay, in, the, school)","(14YOM, student, shot, during, "", horseplay, ""...",suicide,Officer Involved Shooting
2,Unnamed High School,Robbery,Student shot at twice during attempted robbery...,Group of 10 teens attempted to rob 16YOM (Jame...,"(Student, shot, at, twice, during, attempted, ...","(Group, of, 10, teens, attempted, to, rob, 16Y...",suicide,Officer Involved Shooting
3,John F. Kennedy High School,Escalation of Dispute,Argument in school hallway escalated into shoo...,Argument between shooter and victim escalated ...,"(Argument, in, school, hallway, escalated, int...","(Argument, between, shooter, and, victim, esca...",suicide,Officer Involved Shooting
4,David Starr Jordan High School,Unknown,Teens fired at security guard who returned fir...,Group of teens fired shots at the night time s...,"(Teens, fired, at, security, guard, who, retur...","(Group, of, teens, fired, shots, at, the, nigh...",fight,Officer Involved Shooting
5,Pine Bluff Coleman High School,Gang-related,6 gunmen chased student into cafeteria firing ...,6 gunmen chased a student into the cafeteria. ...,"(6, gunmen, chased, student, into, cafeteria, ...","(6, gunmen, chased, a, student, into, the, caf...",fight,Officer Involved Shooting
6,Pierre S. du Pont High School,Escalation of Dispute,Ongoing fight between the students escalated i...,Personal feud between the students escalated i...,"(Ongoing, fight, between, the, students, escal...","(Personal, feud, between, the, students, escal...",fight,Officer Involved Shooting
7,Carver High School,Escalation of Dispute,Shooting after previous fight at pool hall,Shooter and victim had previous fight at pool ...,"(Shooting, after, previous, fight, at, pool, h...","(Shooter, and, victim, had, previous, fight, a...",fight,Officer Involved Shooting
8,Ben Lomond High School,Mental Health,"Showing off gun in hallway, fired two shots st...",Shooter was showing off gun to students in the...,"(Showing, off, gun, in, hallway, ,, fired, two...","(Shooter, was, showing, off, gun, to, students...",fight,Officer Involved Shooting
9,Riverside High School,Escalation of Dispute,Fight between students in parking lot escalate...,Fight between 4 students in the parking lot es...,"(Fight, between, students, in, parking, lot, e...","(Fight, between, 4, students, in, the, parking...",fight,Officer Involved Shooting


'14YOM student shot during "horseplay" in the school hallway. Friend of the victim surrendered to police.'